In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import Imputer

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
#properties = pd.read_csv('trainPreProcess.csv')
#testing = pd.read_csv('testPreProcess.csv')
properties = pd.read_csv('expTrain.csv')
testing = pd.read_csv('testLexico.csv')

In [3]:
properties = properties.loc[:, ['id', 'place_name_encoded', 'lat', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'pileta', 'playroom', 'gimnasio', 'garage', 'excelente', 'lavadero', 'price']]
testing = testing.loc[:, ['id', 'place_name_encoded', 'lat', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'pileta', 'playroom', 'gimnasio', 'garage', 'excelente', 'lavadero']]


In [4]:
#obtengo trainvacios y testvacios con los entries de los respectivos datasets que cumplen:
#en el train set, menos de 10 entradas por place_name_encoded
#en el test set, no tienen entradas en el train set con el mismo place_name_encoded


#l = len(properties['place_name_encoded'].value_counts())
#hardcodeo el valor original (place name encoded va hasta 678)
l=677



trainvacios = properties[properties['place_name_encoded']==1]
testvacios = testing[testing['place_name_encoded']==1]

for x in range(0,l+2):
    if ((len(testing[testing['place_name_encoded']==x])==0) or (len(properties[properties['place_name_encoded']==x])<10)) and (x!=1):
    #if len(testing[testing['place_name_encoded']==x])<10:
    #if len(properties[properties['place_name_encoded']==x])<10:
    
        train0 = properties[properties['place_name_encoded']==x]
        trainvacios = pd.concat([trainvacios, train0])
        
        test0 = testing[testing['place_name_encoded']==x]
        testvacios = pd.concat([testvacios, test0])

In [5]:
trainvacios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19748 entries, 9789 to 53270
Data columns (total 14 columns):
id                       19748 non-null object
place_name_encoded       19748 non-null int64
lat                      19748 non-null float64
surface_covered_in_m2    19748 non-null float64
surface_total_in_m2      19748 non-null float64
floor                    19748 non-null float64
rooms                    19748 non-null float64
pileta                   19748 non-null int64
playroom                 19748 non-null int64
gimnasio                 19748 non-null int64
garage                   19748 non-null int64
excelente                19748 non-null int64
lavadero                 19748 non-null int64
price                    19748 non-null float64
dtypes: float64(6), int64(7), object(1)
memory usage: 2.3+ MB


In [6]:
#los datos restantes tienen más de 10 entradas en train y al menos alguna en test por barrio
#los junto en el dataframe trainaptos

trainaptos= properties[~properties['id'].isin(trainvacios['id'])]
testaptos= testing[~testing['id'].isin(testvacios['id'])]
trainaptos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40252 entries, 0 to 59999
Data columns (total 14 columns):
id                       40252 non-null object
place_name_encoded       40252 non-null int64
lat                      40252 non-null float64
surface_covered_in_m2    40252 non-null float64
surface_total_in_m2      40252 non-null float64
floor                    40252 non-null float64
rooms                    40252 non-null float64
pileta                   40252 non-null int64
playroom                 40252 non-null int64
gimnasio                 40252 non-null int64
garage                   40252 non-null int64
excelente                40252 non-null int64
lavadero                 40252 non-null int64
price                    40252 non-null float64
dtypes: float64(6), int64(7), object(1)
memory usage: 4.6+ MB


In [7]:
#chequeo que quedo bien el split

#la union de trainaptos y trainvacios debe tener misma cantidad de entradas y place_name distintos que propiedades
tra = pd.concat([trainaptos, trainvacios])

print(len(tra), ' = ', len(properties))

print(len(properties['place_name_encoded'].value_counts()), ' = ', len(tra['place_name_encoded'].value_counts()))

#la suma de los place_names distintos en trainvacios y trainaptos debe ser 677
print(len(trainvacios['place_name_encoded'].value_counts()), '+', len(trainaptos['place_name_encoded'].value_counts()))

print()
#AHORA CON TEST
tra = pd.concat([testaptos, testvacios])

print(len(tra), ' = ', len(testing))
print(len(testing['place_name_encoded'].value_counts()), ' = ', len(tra['place_name_encoded'].value_counts()))

print(len(testvacios['place_name_encoded'].value_counts()), '+', len(testaptos['place_name_encoded'].value_counts()))

60000  =  60000
607  =  607
432 + 175

14166  =  14166
223  =  223
48 + 175


In [8]:
trainaptos['place_name_encoded'].value_counts()

450    2425
146    2143
171    1892
490    1569
285    1117
14     1108
338    1086
632    1078
602     994
113     899
483     773
536     767
362     706
487     694
189     670
434     656
37      653
480     609
441     607
188     561
429     528
433     506
503     474
179     473
286     471
603     424
587     408
40      391
387     385
159     362
       ... 
573      23
201      22
303      20
629      19
401      19
393      18
559      18
243      17
12       17
280      17
315      16
230      16
485      16
515      15
448      15
204      15
167      14
638      14
246      13
512      13
575      12
4        12
396      11
218      11
472      11
318      10
606      10
600      10
365      10
332      10
Name: place_name_encoded, Length: 175, dtype: int64

In [9]:
#armo una funcion que devuelva un dataframe con las predicciones KNN del test y train dados y su id

def KNN(test,train,n):
    train_no_price = train.loc[:, ['place_name_encoded', 'lat', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'pileta', 'playroom', 'gimnasio', 'garage', 'excelente', 'lavadero']]
    trainprice = train.loc[:, 'price']
    testnoids = test.loc[:, ['place_name_encoded', 'lat', 'surface_covered_in_m2', 'surface_total_in_m2', 'floor', 'rooms', 'pileta', 'playroom', 'gimnasio', 'garage', 'excelente', 'lavadero']]

    knn = neighbors.KNeighborsRegressor(n, weights="uniform", algorithm="ball_tree")

    predictions = knn.fit(train_no_price, trainprice).predict(testnoids)
    
    predictions = predictions.flatten()
    ids = test.as_matrix()[:,0].flatten()
    results = pd.DataFrame()
    results['id'] = pd.Series(ids.astype('int32'))
    results['price_usd'] = pd.Series(predictions)
    
    
    return results

resultados = KNN(testvacios,trainvacios,25)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
id           114 non-null int32
price_usd    114 non-null float64
dtypes: float64(1), int32(1)
memory usage: 1.4 KB


In [10]:
l=677

for x in range(0,l+2):
    if len(testaptos[testaptos['place_name_encoded']==x])!=0:
        testx = testaptos[testaptos['place_name_encoded']==x]
        trainx = trainaptos[trainaptos['place_name_encoded']==x]
        if len(testaptos[testaptos['place_name_encoded']==x])<25:
            N_neighbors=len(testaptos[testaptos['place_name_encoded']==x])
        else:
            N_neighbors=25
        resx = KNN(testx,trainx,N_neighbors)
        resultados = pd.concat([resultados, resx])

In [11]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166 entries, 0 to 25
Data columns (total 2 columns):
id           14166 non-null int32
price_usd    14166 non-null float64
dtypes: float64(1), int32(1)
memory usage: 276.7 KB


In [12]:
resultados.set_index(['id'], inplace=True)
resultados.to_csv('predicciones_regporplacename_KNN.csv')